In [1]:
import sys

if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = [sys.argv[0]]

In [2]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from src.models import *
from src.constants import *
from src.plotting import *
from src.pot import pot_eval
from src.utils import *
from src.diagnosis import *
from src.merlin import *
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
from time import time
from pprint import pprint

import matplotlib.pyplot as plt
%matplotlib inline

Using backend: pytorch


### Data Preprocessing

In [3]:
def convertNumpy(df):
	x = df[df.columns[3:]].values[::10, :]
	return (x - x.min(0)) / (x.ptp(0) + 1e-4)

In [4]:
dataset_folder = 'data/WADI'

ls = pd.read_csv(os.path.join(dataset_folder, 'WADI_attacklabels.csv'))
train = pd.read_csv(os.path.join(dataset_folder, 'WADI_14days.csv'), skiprows=1000, nrows=2e5)
test = pd.read_csv(os.path.join(dataset_folder, 'WADI_attackdata.csv'))

In [9]:
ls

,Id,Date,Start Time,End Time,Affected
0,1,10/09/2017,19:25:00,19:50:16,1_MV_001
1,2,10/10/2017,10:24:10,10:34:00,1_FIT_001
2,3-4,10/10/2017,10:55:00,11:24:00,"2_LT_002, 1_AIT_001"
3,5,10/10/2017,11:30:40,11:44:50,"2_MCV_101, 2_MCV_201, 2_MCV301, 2_MCV_401, 2_M..."
4,6,10/10/2017,13:39:30,13:50:40,"2_MCV_101, 2_MCV_201"
5,7,10/10/2017,14:48:17,14:59:55,"1_AIT_002, 1_AIT_003, 1_AIT_004, 1_AIT_005, 1_..."
6,8,10/10/2017,17:40:00,17:49:40,2_MCV_007
7,9,10/10/2017,10:55:00,10:56:27,1_P_006
8,10,10/11/2017,11:17:54,11:31:20,1_MV_001
9,11,11/11/2017,11:36:31,11:47:00,2_MCV_007


In [5]:
train.shape

(200000, 130)

In [6]:
test.shape

(172801, 130)

In [7]:
train.head()

,996,9/25/2017,6:16:35.000 PM,174.938,0.559481,11.7524,484.411,0.286654,1.90773,0,...,1.27,1.28,1.29,1.30,1.31,1.32,1.33,63.8467,1.34,0.68
0,997,9/25/2017,6:16:36.000 PM,174.938,0.559481,11.7524,484.411,0.286654,1.90773,0,...,1,1,1,1,1,1,1,63.8467,1,0.68
1,998,9/25/2017,6:16:37.000 PM,174.938,0.559481,11.7524,484.411,0.286654,1.90773,0,...,1,1,1,1,1,1,1,63.8467,1,0.68
2,999,9/25/2017,6:16:38.000 PM,174.213,0.541483,11.7854,482.401,0.286422,2.00360,0,...,1,1,1,1,1,1,1,63.9204,1,0.68
3,1000,9/25/2017,6:16:39.000 PM,174.213,0.541483,11.7854,482.401,0.286422,2.00360,0,...,1,1,1,1,1,1,1,63.9204,1,0.68
4,1001,9/25/2017,6:16:40.000 PM,174.213,0.541483,11.7854,482.401,0.286422,2.00360,0,...,1,1,1,1,1,1,1,63.9204,1,0.68


In [8]:
test.head()

,Row,Date,Time,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_004_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_005_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_001_AL,...,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\LEAK_DIFF_PRESSURE,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\PLANT_START_STOP_LOG,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\TOTAL_CONS_REQUIRED_FLOW
0,1,10/9/2017,6:00:00.000 PM,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,...,1,1,1,1,1,1,1,62.6226,1,0.39
1,2,10/9/2017,6:00:01.000 PM,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,...,1,1,1,1,1,1,1,62.6226,1,0.39
2,3,10/9/2017,6:00:02.000 PM,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,...,1,1,1,1,1,1,1,62.6226,1,0.39
3,4,10/9/2017,6:00:03.000 PM,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,...,1,1,1,1,1,1,1,62.6226,1,0.39
4,5,10/9/2017,6:00:04.000 PM,164.21,0.529486,11.9972,482.48,0.331167,0.001273,0,...,1,1,1,1,1,1,1,62.6226,1,0.39


In [10]:
train.dropna(how='all', inplace=True); test.dropna(how='all', inplace=True)
train.fillna(0, inplace=True); test.fillna(0, inplace=True)

In [11]:
test['Time'] = test['Time'].astype(str)
test['Time'] = pd.to_datetime(test['Date'] + ' ' + test['Time'])

In [13]:
labels = test.copy(deep = True)
for i in test.columns.tolist()[3:]:
    labels[i] = 0

In [15]:
labels.head()

,Row,Date,Time,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_004_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_005_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_001_AL,...,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\LEAK_DIFF_PRESSURE,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\PLANT_START_STOP_LOG,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\TOTAL_CONS_REQUIRED_FLOW
0,1,10/9/2017,2017-10-09 18:00:00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,10/9/2017,2017-10-09 18:00:01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,10/9/2017,2017-10-09 18:00:02,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,10/9/2017,2017-10-09 18:00:03,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,10/9/2017,2017-10-09 18:00:04,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
for i in ['Start Time', 'End Time']: 
	ls[i] = ls[i].astype(str)
	ls[i] = pd.to_datetime(ls['Date'] + ' ' + ls[i])

In [17]:
for index, row in ls.iterrows():
	to_match = row['Affected'].split(', ')
	matched = []
	for i in test.columns.tolist()[3:]:
		for tm in to_match:
			if tm in i: 
				matched.append(i); break			
	st, et = str(row['Start Time']), str(row['End Time'])
	labels.loc[(labels['Time'] >= st) & (labels['Time'] <= et), matched] = 1

In [18]:
train, test, labels = convertNumpy(train), convertNumpy(test), convertNumpy(labels)

In [19]:
print(train.shape, test.shape, labels.shape)

(20000, 127) (17281, 127) (17281, 127)


### Training

In [20]:
def convert_to_windows(data, model):
	windows = []; w_size = model.n_window
	for i, g in enumerate(data): 
		if i >= w_size: w = data[i-w_size:i]
		else: w = torch.cat([data[0].repeat(w_size-i, 1), data[0:i]])
		windows.append(w)
	return torch.stack(windows)

def data_loader():
	global train, test, labels
	
	train_loader = DataLoader(train, batch_size=train.shape[0])
	test_loader = DataLoader(test, batch_size=test.shape[0])
	labels_loader = labels
	return train_loader, test_loader, labels_loader

def create_model(modelname, dims):
	import src.models
	model_class = getattr(src.models, modelname)
	model = model_class(dims).double()

	optimizer = torch.optim.AdamW(model.parameters() , lr=model.lr, weight_decay=1e-5)
	scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.9)

	print(f"{color.GREEN}Creating new model: {model.name}{color.ENDC}")
	
	epoch = -1
	accuracy_list = []

	return model, optimizer, scheduler, epoch, accuracy_list

def backprop(epoch, model, data, dataO, optimizer, scheduler, training = True):
    feats = dataO.shape[1]
    l = nn.MSELoss(reduction = 'none')
    
    data_x = torch.DoubleTensor(data); dataset = TensorDataset(data_x, data_x)
    bs = model.batch if training else len(data)
    dataloader = DataLoader(dataset, batch_size = bs)
    n = epoch + 1; w_size = model.n_window
    l1s, l2s = [], []
    if training:
        for d, _ in dataloader:
            local_bs = d.shape[0]
            window = d.permute(1, 0, 2)
            elem = window[-1, :, :].view(1, local_bs, feats)
            z = model(window, elem)
            l1 = l(z, elem) if not isinstance(z, tuple) else (1 / n) * l(z[0], elem) + (1 - 1/n) * l(z[1], elem)
            if isinstance(z, tuple): z = z[1]
            l1s.append(torch.mean(l1).item())
            loss = torch.mean(l1)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        scheduler.step()
        tqdm.write(f'Epoch {epoch},\tL1 = {np.mean(l1s)}')
        return np.mean(l1s), optimizer.param_groups[0]['lr']
    else:
        for d, _ in dataloader:
            window = d.permute(1, 0, 2)
            elem = window[-1, :, :].view(1, bs, feats)
            z = model(window, elem)
            if isinstance(z, tuple): z = z[1]
        loss = l(z, elem)[0]
        return loss.detach().numpy(), z.detach().numpy()[0]

In [21]:
train_loader, test_loader, labels_loader = data_loader()
model, optimizer, scheduler, epoch, loss_list = create_model('TranAD', labels_loader.shape[1])

trainD, testD = next(iter(train_loader)), next(iter(test_loader))
trainO, testO = trainD, testD 

trainD, testD = convert_to_windows(trainD, model), convert_to_windows(testD, model)

Creating new model: TranAD


In [22]:
print(f'{color.HEADER}Training TranAD on WADI dataset.{color.ENDC}')
num_epochs = 5; e = epoch + 1; start = time()
for e in tqdm(list(range(epoch+1, epoch+num_epochs+1))):
    lossT, lr = backprop(e, model, trainD, trainO, optimizer, scheduler)
    loss_list.append((lossT, lr))
print(color.BOLD+'Training time: '+"{:10.4f}".format(time()-start)+' s'+color.ENDC)

  0%|          | 0/5 [00:00<?, ?it/s]

Training TranAD on WADI dataset.


 20%|██        | 1/5 [00:53<03:35, 53.95s/it]

Epoch 0,	L1 = 0.046638660847307764


 40%|████      | 2/5 [01:44<02:36, 52.18s/it]

...

Epoch 2,	L1 = 0.017521387380686298


 80%|████████  | 4/5 [03:32<00:53, 53.13s/it]

Epoch 3,	L1 = 0.01318711861678395


100%|██████████| 5/5 [04:24<00:00, 52.96s/it]

Epoch 4,	L1 = 0.010754167436603543
Training time:   264.8138 s


### Testing

In [23]:
torch.zero_grad = True
model.eval()
print(f'{color.HEADER}Testing TranAD on WADI dataset{color.ENDC}')
loss, y_pred = backprop(0, model, testD, testO, optimizer, scheduler, training=False)

### Scores
df = pd.DataFrame()
lossT, _ = backprop(0, model, trainD, trainO, optimizer, scheduler, training=False)
for i in range(loss.shape[1]):
    lt, l, ls = lossT[:, i], loss[:, i], labels_loader[:, i]
    result, pred = pot_eval(lt, l, ls); preds.append(pred)
    df = df.append(result, ignore_index=True)

lossTfinal, lossFinal = np.mean(lossT, axis=1), np.mean(loss, axis=1)
labelsFinal = (np.sum(labels_loader, axis=1) >= 1) + 0
result, _ = pot_eval(lossTfinal, lossFinal, labelsFinal)
result.update(hit_att(loss, labels_loader))
result.update(ndcg(loss, labels_loader))
print(df)
pprint(result)

Testing TranAD on WADI dataset


c:\Users\Nirajan\Desktop\Fusemachines\TranAD\env\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Nirajan\Desktop\Fusemachines\TranAD\env\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


             FN            FP  ROC/AUC            TN         TP        f1  \
0    174.982502      1.000000      0.0  17105.017498   0.000000  0.000000   
1      0.000000      4.006999      0.0  17207.000000  69.993001  0.972167   
2      0.000000  11029.006999      0.0   6182.000000  69.993001  0.012533   
3     69.993001   1079.000000      0.0  16132.006999   0.000000  0.000000   
4     69.993001    175.000000      0.0  17036.006999   0.000000  0.000000   
..          ...           ...      ...           ...        ...       ...   
122    0.000000      0.000000      0.0  17281.000000   0.000000  0.000000   
123    0.000000     66.000000      0.0  17215.000000   0.000000  0.000000   
124    0.000000     17.000000      0.0  17264.000000   0.000000  0.000000   
125    0.000000     34.000000      0.0  17247.000000   0.000000  0.000000   
126    0.000000      9.000000      0.0  17272.000000   0.000000  0.000000   

     precision  recall  threshold  
0     0.000000     0.0   0.701500  
1  